
## Without Zeeman and with orbital effects, spin-orbit has little effect on $I_c$

## With spin-orbit and with orbital effects, Zeeman has little effect on $I_c$

## Without orbital effects and without spin-orbit, the $B$ dependence scales both with $L$ and $\mu$

## In single mode regime, Zeeman has the predominant effect on $I_c$

## Both in the single mode and multimode regime, the orbital effect has the predominant effect on $I_c$

## $0-\pi$ transitions?
* $L=640, \mu=50$
* $L=640, \mu=40$
* $L=160, \mu=30$


In [ ]:
import matplotlib.pylab as plt
import matplotlib.patches as patches
import numpy as np
import pandas as pd
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

golden_mean = (np.sqrt(5) - 1) / 2 # Aesthetic ratio
fig_width_pt = 246.0 # Columnwidth
inches_per_pt = 1.0 / 72.27 # Convert pt to inches
fig_width = fig_width_pt * inches_per_pt
fig_height = fig_width * golden_mean # height in inches
fig_size = [fig_width, fig_height]

params = {'backend': 'ps',
          'axes.labelsize': 10,
          'font.size': 10,
          'legend.fontsize': 8,
          'xtick.labelsize': 8,
          'ytick.labelsize': 8,
#           'text.usetex': True,
          'figure.figsize': fig_size,
          'font.family': 'serif',
          'font.serif': 'Computer Modern Roman',
          'legend.frameon': True,
          'savefig.dpi': 1000
         }

plt.rcParams.update(params)
# plt.rc('text.latex', preamble=[r'\usepackage{color}', r'\usepackage{bm}'])

# Multi mode regime:

In [ ]:
df = pd.read_hdf('data/all_data_circular_A_in_SM_and_SC.hdf')
gb = df.groupby(['T', 'L', 'with_leads', 'orbital', 'g', 'alpha', 'mu'])

In [ ]:
total = gb.get_group((1, 640, True, True, 50, 20, 50))
Zeeman_only = gb.get_group((1, 640, True, False, 50, 20, 50))
orbital_only = gb.get_group((1, 640, True, True, 0, 20, 50))
no_spin_orbit = gb.get_group((1, 640, True, False, 50, 0, 50))

fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_axes([0.125, 0.2, (0.95-0.125), (0.95-0.2)])
ax.plot(total['B_x'], total['current_c'], label='all effects')
ax.plot(Zeeman_only['B_x'], Zeeman_only['current_c'], label='$A=0$')
ax.plot(orbital_only['B_x'], orbital_only['current_c'], label='$g=0$')
ax.plot(no_spin_orbit['B_x'], no_spin_orbit['current_c'], label=r'$\alpha=0, \; A=0$')
ax.set_yticks(np.arange(0, 100, 10))
ax.set_ylim(0, 50)
ax.set_xlabel('Magnetic field $B$ [$T$]')
ax.set_ylabel('Critical current $I_c$ [meV$/\Phi_0$]')
ax.legend(framealpha=0.2)
# fig.savefig('decay_length_2d.pdf', format='pdf', dpi=1000)
plt.show()

# Single mode regime:

In [ ]:
total = gb.get_group((1, 640, True, True, 50, 20, 10))
Zeeman_only = gb.get_group((1, 640, True, False, 50, 20, 10))
orbital_only = gb.get_group((1, 640, True, True, 0, 20, 10))
no_spin_orbit = gb.get_group((1, 640, True, False, 50, 0, 10))

fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_axes([0.125,0.2,(0.95-0.125),(0.95-0.2)])
ax.plot(total['B_x'], total['current_c'], label='all effects')
ax.plot(Zeeman_only['B_x'], Zeeman_only['current_c'], label='$A=0$')
ax.plot(orbital_only['B_x'], orbital_only['current_c'], label='$g=0$')
ax.plot(no_spin_orbit['B_x'], no_spin_orbit['current_c'], label=r'$\alpha=0, \; A=0$')
ax.set_yticks(np.arange(0, 10, 0.2))
ax.set_ylim(0, 0.35)
ax.set_xlabel('Magnetic field $B$ [$T$]')
ax.set_ylabel('Critical current $I_c$ [meV$/\Phi_0$]')
ax.legend(framealpha=0.2)
# fig.savefig('decay_length_2d.pdf', format='pdf', dpi=1000)
plt.show()

# Gate dependence

In [ ]:
df = pd.read_hdf('data/all_gate_data_current_as_function_of_gate_and_B_x.hdf')
gb = df.groupby(['B_x', 'gate_size'])
gr = gb.get_group((1, 640))

fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_axes([0.125,0.2,(0.95-0.125),(0.95-0.2)])
ax.plot(gr['Vs'], gr['current_c'])

In [ ]:
df = pd.read_hdf('data/all_gate_data_current_as_function_of_gate_and_B_x.hdf')
gb = df.groupby(['gate_size'])
gr = gb.get_group((640))

fig = plt.figure(figsize=(fig_width, fig_height))
ax = fig.add_axes([0.125,0.2,(0.95-0.125),(0.95-0.2)])
ax.imshow(gr['current_c'])